In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.cluster.hierarchy import fcluster, set_link_color_palette
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.mixture import GaussianMixture as GM
import dc_stat_think as dcst
import dabest
sep = '/'
plt.style.use('ggplot')
plt.rcParams['axes.facecolor']='#f2efef'

In [2]:
font = {'fontname':'Calibri'}

### Load csv, index out citalopram recordings

In [3]:
recording = r'E:\CIT_WAY\dat_files\cat\csvs'
filename ='neuron_stats.csv'
figpath = r'C:\Users\ASUS\Desktop\Images\Clustered Scatterplot'
figname = 'Chronic Clusters.png'

In [4]:
df = pd.read_csv(os.sep.join([recording, filename]))
df.index.name = 'Cluster_ID'
df = df[pd.notnull(df['SW_troff'])]

In [5]:
df_cit = df[(df['recording']=='Chronic_04_2018-08-05')|(df['recording']=='Chronic_30_2018-07-28')|(df['recording']=='Chronic_40_2018-08-13')|(df['recording']=='Chronic_41_2018-08-09')]
df_cit = df_cit[(df_cit['CV ISI'] != 0)&(df_cit['Firing Rate'] != 0)]

### Scale and label data with GMM

In [ ]:
scale =RobustScaler()
scaled_data = scale.fit_transform(df_cit[['CV ISI', 'Firing Rate']])

In [ ]:
gmm = GM(n_components=3, covariance_type='diag').fit(scaled_data)
GMM_labels = gmm.predict(scaled_data)
df_cit['GMM_labels']=list(GMM_labels)

In [ ]:
sns.lmplot(data=df_cit, y='Firing Rate', x='CV ISI', fit_reg=False, legend= False, hue='GMM_labels', scatter_kws={'linewidths':1,'edgecolor':'grey', 's':500, 'alpha':0.8},palette=['#ee4035', '#0392cf', '#7bc043'], size=(20))
plt.xlabel('CV ISI', **font, fontsize=90)
plt.ylabel('Firing Rate [Hz]', **font, fontsize=90)
plt.yticks(np.arange(0, 45, 5), **font, fontsize=80)
plt.xticks(np.arange(0, 4, 0.5), **font, fontsize=80, rotation=70)
plt.legend(['Fast Firing', 'Slow Regular', 'Slow Irregular'], prop={'size':50})
plt.tight_layout()
#plt.savefig(s.sep.join([figpath, figname]), dpi=600)
plt.show()

# Plot using arbitrary cutoff

#### Preferably this has been carried out using preprocess script

In [ ]:
def neuron_category_mapper(row):

    if (row['Firing Rate'] <= 10) & (row['CV ISI'] <= 0.8):
        rate = 'slow'
        reg = 'regular'
    elif (row['Firing Rate'] <= 10) & (row['CV ISI'] >= 0.8):
        rate = 'slow'
        reg = 'irregular'
    elif (row['Firing Rate'] > 10) & (row['Firing Rate'] <25):
        rate= 'fast'
        reg = 'firing'
    else:
        rate= 'V.fast'
        reg = 'firing'
    return ' '.join([rate, reg])


In [ ]:
df_cit['category'] = df_cit.apply(neuron_category_mapper, axis=1)

### Plot the arbitrary-labelled dataset

In [ ]:
sns.lmplot(data=df_cit, y='Firing Rate', x='CV ISI', fit_reg=False, legend= False, hue='category', scatter_kws={'linewidths':1,'edgecolor':'grey', 's':500, 'alpha':0.8},palette=['#CCCC00','#7bc043', '#ee4035','#0392cf'], size=(20))
plt.xlabel('CV ISI', **font, fontsize=90)
plt.ylabel('Firing Rate [Hz]', **font, fontsize=90)
plt.yticks(**font, fontsize=80)
plt.xticks(**font, fontsize=80, rotation=70)
plt.legend(['V.Fast Firing', 'Slow Irregular', 'Slow Regular', 'Fast Firing'], prop={'size':50})
plt.tight_layout()
#plt.savefig(r'C:\Users\ASUS\Desktop\Images\Clustered Scatterplot\Saline Clusters.png', dpi=600)
plt.show()

# Scatter plot with chosen clusters marked with a star

### To find which cluster to mark, use the plotly plot in another notebook

In [ ]:
df_chosen_cluster = df[(df['spike_cluster'] == 154) | (df['spike_cluster'] == 147) | (df['spike_cluster'] == 159) |(df['spike_cluster'] == 98) & (df['recording'] == '2018-05-20_05')]

In [ ]:
path = r'C:\Users\ASUS\Desktop\Images'
sns.lmplot(data=df, y='Firing Rate', x='CV ISI', fit_reg=False, legend= False, hue='GMM_labels', scatter_kws={'linewidths':1,'edgecolor':'grey', 's':500, 'alpha':0.7}, size=(20),  palette=['#0392cf', '#CCCC00', '#7bc043', '#ee4035'])
plt.scatter(x=df_chosen_cluster['CV ISI'], y=df_chosen_cluster['Firing Rate'], c=df_chosen_cluster['colors'], alpha=1, s=3500, edgecolors='k', linewidths=1.5, marker='*')
#plt.title('Scatterplot of Hierarchical Clustering', **font, fontsize=60)
plt.xlabel('CV ISI', **font, fontsize=60)
plt.ylabel('Firing Rate [Hz]', **font, fontsize=60)
plt.yticks(**font, fontsize=50)
plt.xticks(**font, fontsize=50, rotation=70)
plt.legend(['Fast Firing','V.Fast Firing', 'Slow Irregular', 'Slow Regular'], prop={'size':50})
plt.tight_layout()
#plt.savefig(os.sep.join([path, 'Clustered Scatterplot', 'Clustered Scatterplot.png']), dpi=600)
plt.show()


### Generate the summary table for different neuron types

The stats included are count, mean, std, min, and max for CV ISI and Firing Rate

In [ ]:
df_cit.groupby('category')[['CV ISI', 'Firing Rate']].describe().loc[:, (slice('CV ISI', 'Firing Rate'), ['count', 'mean', 'std', 'min', 'max'])].transpose()